<b> LOADING THE DATA <b>

In [0]:
dataset_year = 2020

In [0]:
# DO NOT CHANGE THE CONTENT OF THIS CELL
import os
import sys

fileroot = "clinicaltrial_" + str(dataset_year) + "_csv"
renamed_fileroot = "clinicaltrial_" + str(dataset_year) + ".csv"
mesh_csv = "/FileStore/tables/mesh.csv"
pharma_csv = "/FileStore/tables/pharma.csv"

# if 'dbruntime.dbutils' in sys.modules.keys():
try:
    dbutils.fs.ls("/FileStore/tables/" + renamed_fileroot)
except:
    dbutils.fs.cp("/FileStore/tables/" + fileroot + ".gz", "file:/tmp/")
    os.environ['fileroot'] = fileroot

In [0]:
%sh
gunzip /tmp/ /tmp/$fileroot.gz

gzip: /tmp/ is a directory -- ignored
gzip: /tmp/.gz.gz: No such file or directory


In [0]:
try:
    dbutils.fs.ls("file:/tmp/" + fileroot)
    dbutils.fs.mv("file:/tmp/" + fileroot, "/FileStore/tables/" + renamed_fileroot, True)
except:
    pass

In [0]:
dbutils.fs.ls("/FileStore/tables/")

Out[8]: [FileInfo(path='dbfs:/FileStore/tables/accounts.zip', name='accounts.zip', size=5297592),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019.csv', name='clinicaltrial_2019.csv', size=42400056),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2019_csv.gz', name='clinicaltrial_2019_csv.gz', size=10060669),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020.csv', name='clinicaltrial_2020.csv', size=46318151),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2020_csv.gz', name='clinicaltrial_2020_csv.gz', size=10981608),
 FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021_csv.gz', name='clinicaltrial_2021_csv.gz', size=11921810),
 FileInfo(path='dbfs:/FileStore/tables/devicestatus.zip', name='devicestatus.zip', size=23873574),
 FileInfo(path='dbfs:/FileStore/tables/json_activations.zip', name='json_activations.zip', size=8411369),
 FileInfo(path='dbfs:/FileStore/tables/letter_frequencies.txt', name='letter_frequencies.txt', size=2593894),
 FileInfo(path

In [0]:
# fileroot = "clinicaltrial_" + str(dataset_year) + ".csv"

clinical_csv = "/FileStore/tables/" + renamed_fileroot
mesh_csv = "/FileStore/tables/mesh.csv"
pharma_csv = "/FileStore/tables/pharma.csv"

# clinical_csv

<b>PREPARE THE DATA<b>

USING DATAFRAME

In [0]:
from pyspark.sql.functions import *

In [0]:
clinicalDF = spark.read.option("header","true").option("inferSchema", "true").option("delimiter", '|').csv(clinical_csv)
clinicalDF.show(10)

+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|         Id|             Sponsor|              Status|   Start|Completion|                Type|Submission|          Conditions|       Interventions|
+-----------+--------------------+--------------------+--------+----------+--------------------+----------+--------------------+--------------------+
|NCT02758028|The University of...|          Recruiting|Aug 2005|  Nov 2021|      Interventional|  Apr 2016|                null|                null|
|NCT02751957|     Duke University|           Completed|Jul 2016|  Jul 2020|      Interventional|  Apr 2016|Autistic Disorder...|                null|
|NCT02758483|Universidade Fede...|           Completed|Mar 2017|  Jan 2018|      Interventional|  Apr 2016|   Diabetes Mellitus|                null|
|NCT02759848|Istanbul Medeniye...|           Completed|Jan 2012|  Dec 2014|       Observational|  Ma

In [0]:
meshDF = spark.read.option("header","true").option("inferSchema", "true").csv(mesh_csv)
meshDF.show(10)

+----------+-------------------+
|      term|               tree|
+----------+-------------------+
|Calcimycin|D03.633.100.221.173|
|   A-23187|D03.633.100.221.173|
|   Temefos|D02.705.400.625.800|
|   Temefos|D02.705.539.345.800|
|   Temefos|D02.886.300.692.800|
|     Abate|D02.705.400.625.800|
|     Abate|D02.705.539.345.800|
|     Abate|D02.886.300.692.800|
|     Difos|D02.705.400.625.800|
|     Difos|D02.705.539.345.800|
+----------+-------------------+
only showing top 10 rows



In [0]:
pharmaDF = spark.read.option("header","true").option("inferSchema", "true").csv(pharma_csv)
display(pharmaDF)

Company,Parent_Company,Penalty_Amount,Subtraction_From_Penalty,Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting,Penalty_Year,Penalty_Date,Offense_Group,Primary_Offense,Secondary_Offense,Description,Level_of_Government,Action_Type,Agency,Civil/Criminal,Prosecution_Agreement,Court,Case_ID,Private_Litigation_Case_Title,Lawsuit_Resolution,Facility_State,City,Address,Zip,NAICS_Code,NAICS_Translation,HQ_Country_of_Parent,HQ_State_of_Parent,Ownership_Structure,Parent_Company_Stock_Ticker,Major_Industry_of_Parent,Specific_Industry_of_Parent,Info_Source,Notes
Abbott Laboratories,Abbott Laboratories,"$5,475,000",$0,"$5,475,000",2013,20131227,government-contracting-related offenses,False Claims Act and related,kickbacks and bribery,"Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the company's carotid, biliary and peripheral vascular products.",federal,agency action,Justice Department Civil Division,civil,null,null,null,null,null,null,null,null,null,null,null,USA,Illinois,publicly traded,ABT,pharmaceuticals,pharmaceuticals,https://www.justice.gov/opa/pr/abbott-laboratories-pays-us-5475-million-settle-claims-company-paid-kickbacks-physicians,null
Abbott Laboratories Inc.,AbbVie,"$1,500,000,000",$0,"$1,500,000,000",2012,20120507,healthcare-related offenses,off-label or unapproved promotion of medical products,null,Global Health Care Company Abbott Laboratories Inc. has pleaded guilty and agreed to pay $1.5 billion to resolve its criminal and civil liability arising from the company's unlawful promotion of the prescription drug Depakote for uses not approved as safe and effective by the Food and Drug Administration. The resolution - the second largest payment by a drug company - includes a criminal fine and forfeiture totaling $700 million and civil settlements with the federal government and the states totaling $800 million. Abbott also will be subject to court-supervised probation and reporting obligations for Abbotts CEO and Board of Directors.,federal,agency action,Food and Drug Administration referral to the Justice Department,civil and criminal,null,null,null,null,null,null,null,null,null,null,null,USA,Illinois,publicly traded,ABBV,pharmaceuticals,pharmaceuticals,http://www.justice.gov/opa/pr/abbott-labs-pay-15-billion-resolve-criminal-civil-investigations-label-promotion-depakote,null
Abbott Laboratories Inc.,AbbVie,"$126,500,000",$0,"$126,500,000",2010,20101207,government-contracting-related offenses,False Claims Act and related,null,"Abbott Laboratories Inc., B. Braun Medical Inc. and Roxane Laboratories Inc. n/k/a Boehringer Ingelheim Roxane Inc. and affiliated entities agreed to pay $421 million to settle False Claims Act allegations, that they engaged in a scheme to report false and inflated prices for numerous pharmaceutical products knowing that federal healthcare programs relied on those reported prices to set payment rates.",federal,agency action,Justice Department Civil Division,civil,null,null,null,null,null,null,null,null,null,null,null,USA,Illinois,publicly traded,ABBV,pharmaceuticals,pharmaceuticals,https://www.justice.gov/opa/pr/pharmaceutical-manufacturers-pay-4212-million-settle-false-claims-act-cases,null
"Abbott Laboratories Puerto Rico, Inc.",Abbott Laboratories,"$49,045",$0,"$49,045",2009,20090305,employment-related offenses,wage and hour violation,Fair Labor Standards Act,null,federal,agency action,Labor Department Wage and Hour Division,civil,null,null,1527129,null,null,Puerto Rico,San Juan,null,null,423450,"423450: Medical, Dental, and Hospital Equipment and Supplies Merchant Wholesalers",USA,Illinois,publicly traded,ABT,pharmaceuticals,pharmaceuticals,"March 7, 2017 download of a dataset posted by the Wage and Hour Division at https://enforcedata.dol.gov/views/data_summary.php","Date and year are the Findings End Date in the dataset posted by the Wage and Hour Division, which does not provide ca

<b>ANALYSE THE DATA<b>

In [0]:
# QUESTION1 - The number of studies in the dataset
clinicalDF.distinct().count()

Out[14]: 356466

In [0]:
# QUESTION2 - List all the Type of studies in the dataset along with the frequencies of each type
typesFromDF = clinicalDF.groupBy("Type").count().orderBy("count", ascending=False)
typesFromDF.show()

+--------------------+------+
|                Type| count|
+--------------------+------+
|      Interventional|277631|
|       Observational| 71434|
|Observational [Pa...|  7332|
|     Expanded Access|    69|
+--------------------+------+



In [0]:
# QUESTION3 - The top 5 Conditions with their frequencies
splitConditionsFromDF = split(regexp_replace(col("Conditions"), "(^\[)|(\]$)", ""), ",")
explodeConditionsFromDF = clinicalDF.withColumn(
    "Conditions", explode(splitConditionsFromDF)
)

topConditionsFromDF = explodeConditionsFromDF.groupBy("Conditions").count().orderBy("count", ascending=False)
topConditionsFromDF.show(5, truncate=False)

+-----------------+-----+
|Conditions       |count|
+-----------------+-----+
|Carcinoma        |12245|
|Diabetes Mellitus|10425|
|Neoplasms        |8534 |
|Breast Neoplasms |8009 |
|Syndrome         |7419 |
+-----------------+-----+
only showing top 5 rows



In [0]:
# QUESTION4 - The 5 most frequent roots from the hierarchy codes
rootsFromDF = meshDF.join(explodeConditionsFromDF, meshDF.term == explodeConditionsFromDF.Conditions).\
                    select("tree", "term", "Conditions").withColumn('tree', substring('tree', 1,3)).\
                    groupBy("tree").count().orderBy("count", ascending=False)
rootsFromDF.show(10)

+----+------+
|tree| count|
+----+------+
| C04|133091|
| C23|124589|
| C01| 94293|
| C14| 88065|
| C10| 83894|
| C06| 79382|
| C08| 61138|
| C13| 39316|
| C18| 38439|
| C12| 37294|
+----+------+
only showing top 10 rows



In [0]:
# QUESTION 5 - The 10 most common sponsors that are not pharmaceutical companies with the number of clinical trials they have sponsored
nonPharmaSponsorsDF = clinicalDF.join(pharmaDF, pharmaDF.Parent_Company == clinicalDF.Sponsor, "left").filter(pharmaDF.Parent_Company.isNull())
mostCommonNonPharmaSponsorsDF = nonPharmaSponsorsDF.filter(nonPharmaSponsorsDF.Status!="Active").groupBy("Sponsor").count().orderBy("count", ascending=False)

mostCommonNonPharmaSponsorsDF.show(10, truncate=False)

+---------------------------------------+-----+
|Sponsor                                |count|
+---------------------------------------+-----+
|National Cancer Institute (NCI)        |3100 |
|M.D. Anderson Cancer Center            |2238 |
|Merck Sharp & Dohme Corp.              |2184 |
|Mayo Clinic                            |2097 |
|Assistance Publique - Hôpitaux de Paris|2043 |
|Novartis Pharmaceuticals               |1962 |
|Massachusetts General Hospital         |1823 |
|Assiut University                      |1806 |
|Hoffmann-La Roche                      |1761 |
|National Taiwan University Hospital    |1720 |
+---------------------------------------+-----+
only showing top 10 rows



In [0]:
# QUESTION 6 - Number of completed studies each month in a given year
# yearfilter = "%" + str(dataset_year)

print(dataset_year)
# completedStudiesDF = clinicalDF.filter(clinicalDF.Status=="Completed").filter(col("Completion").like(yearfilter)).\
#                                     withColumn('Completion', substring('Completion', 1,3)).groupBy("Completion").count()
# numberOfCompletedStudiesDF = completedStudiesDF.sort(unix_timestamp(col("Completion"),"MMM"))

# numberOfCompletedStudiesDF.show()

2020


<b>VISUALISE RESULTS<b>

In [0]:
# Q2: Install bokeh
%pip install bokeh

In [0]:
# Values to Plot on Bar chart
month = numberOfCompletedStudiesDF.select("Completion").rdd.map(lambda row: row.Completion).collect()
counts = numberOfCompletedStudiesDF.select("count").rdd.map(lambda row: row["count"]).collect()

In [0]:
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN

p = figure(x_range=month, height=250, title="Completed studies each month in a given year",
           toolbar_location=None, tools="")

p.vbar(x=completionMonth, top=counts, width=0.7)

p.xgrid.grid_line_color = None
p.y_range.start = 0

html = file_html(p, CDN, "plot")
displayHTML(html)

USING RDD